In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [5]:
CLIENT_ID = '5FTEJX53IQQCNFVLBL5WWE1D4PSK4044TIBLTPYRJD5BM3BL' # your Foursquare ID
CLIENT_SECRET = 'V3TGZIFZERVYDMZOEJI02RCVOQV5URZQPJW4GA2EYN0SDIJN' # your Foursquare Secret
ACCESS_TOKEN = 'RSVIE11T0P5YCUKN1VWMJDXMASOFQP1RCP5PRK0Y4JWGHILT' # your FourSquare Access Token
VERSION = '20201223'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5FTEJX53IQQCNFVLBL5WWE1D4PSK4044TIBLTPYRJD5BM3BL
CLIENT_SECRET:V3TGZIFZERVYDMZOEJI02RCVOQV5URZQPJW4GA2EYN0SDIJN


In [6]:
!pip -q install folium
import folium

In [7]:
address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [8]:
latitude = 43.6534817
longitude = -79.3839347

In [9]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius=500&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=5FTEJX53IQQCNFVLBL5WWE1D4PSK4044TIBLTPYRJD5BM3BL&client_secret=V3TGZIFZERVYDMZOEJI02RCVOQV5URZQPJW4GA2EYN0SDIJN&ll=43.6534817,-79.3839347&v=20201223&radius=500&limit=30'

In [10]:
import requests

In [12]:
results = requests.get(url).json()

In [13]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5227bb01498e17bf485e6202',
  'name': 'Downtown Toronto',
  'location': {'lat': 43.65323167517444,
   'lng': -79.38529600606677,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.65323167517444,
     'lng': -79.38529600606677}],
   'distance': 113,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['Toronto ON', 'Canada']},
  'categories': [{'id': '4f2a25ac4b909258e854f55f',
    'name': 'Neighborhood',
    'pluralName': 'Neighborhoods',
    'shortName': 'Neighborhood',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-5227bb01498e17bf485e6202-0'}

In [14]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
#dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_categories_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,neighborhood,id
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296,"[{'label': 'display', 'lat': 43.65323167517444...",113,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,NaN,NaN,5227bb01498e17bf485e6202
1,Nathan Phillips Square,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",43.652270,-79.383516,"[{'label': 'display', 'lat': 43.65227047322295...",138,CA,Toronto,ON,Canada,"[100 Queen St W (at Bay St), Toronto ON M5H 2N...",100 Queen St W,at Bay St,M5H 2N1,NaN,4ad4c05ef964a520a6f620e3
2,Poke Guys,"[{'id': '5bae9231bedf3950379f89d4', 'name': 'P...",43.654895,-79.385052,"[{'label': 'display', 'lat': 43.65489527525682...",181,CA,Toronto,ON,Canada,"[112 Elizabeth St (at Dundas St W), Toronto ON...",112 Elizabeth St,at Dundas St W,M5G 1P5,NaN,57bcd3b7498e652a678d0378
3,Indigo,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.653515,-79.380696,"[{'label': 'display', 'lat': 43.65351471121164...",260,CA,Toronto,ON,Canada,"[220 Yonge St, Toronto ON M5B 2H1, Canada]",220 Yonge St,NaN,M5B 2H1,Downtown Yonge,4b2a6eb8f964a52012a924e3
4,Chatime 日出茶太,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",43.655542,-79.384684,"[{'label': 'display', 'lat': 43.65554164147378...",237,CA,Toronto,ON,Canada,"[132 Dundas St W (btwn Bay & University), Toro...",132 Dundas St W,btwn Bay & University,M5G 1C3,NaN,4e2284b11fc7c0ef9857d143
5,Textile Museum of Canada,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",43.654396,-79.386500,"[{'label': 'display', 'lat': 43.65439630500274...",230,CA,Toronto,ON,Canada,[55 Centre Avenue (University Ave. and Dundas ...,55 Centre Avenue,University Ave. and Dundas St W.,M5G 2H5,NaN,4ad4c05ef964a520e2f620e3
6,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",43.654447,-79.380952,NaN,263,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",220 Yonge St,btwn Queen & Dundas,M5B 2H1,"Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3
7,Japango,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",43.655268,-79.385165,"[{'label': 'display', 'lat': 43.65526771691681...",222,CA,Toronto,ON,Canada,"[122 Elizabeth St. (at Dundas St. W), Toronto ...",122 Elizabeth St.,at Dundas St. W,M5G 1P5,NaN,4ae7b27df964a52068ad21e3
8,Old City Hall,"[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",43.652009,-79.381744,"[{'label': 'display', 'lat': 43.65200880087612...",240,CA,Toronto,ON,Canada,"[60 Queen Street West, Toronto ON M5H 1A1, Can...",60 Queen Street West,NaN,M5H 1A1,NaN,4ad4c05ef964a5208ef620e3
9,LUSH,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",43.653557,-79.380400,"[{'label': 'display', 'lat': 43.653557, 'lng':...",284,CA,Toronto,ON,Canada,"[220 Yonge St, Unit B215-A (in Toronto Eaton C...","220 Yonge St, Unit B215-A",in Toronto Eaton Centre,M5B 2H1,NaN,4bd0b30d41b9ef3b8fa0fae5


In [15]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Toronto


# add name as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Toronto',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Poke Guys,Poke Place,43.654895,-79.385052
3,Indigo,Bookstore,43.653515,-79.380696
4,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
dataframe_filtered.head(100)

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,crossStreet,postalCode,neighborhood,id
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296,"[{'label': 'display', 'lat': 43.65323167517444...",113,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,NaN,NaN,5227bb01498e17bf485e6202
1,Nathan Phillips Square,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",43.652270,-79.383516,"[{'label': 'display', 'lat': 43.65227047322295...",138,CA,Toronto,ON,Canada,"[100 Queen St W (at Bay St), Toronto ON M5H 2N...",100 Queen St W,at Bay St,M5H 2N1,NaN,4ad4c05ef964a520a6f620e3
2,Poke Guys,"[{'id': '5bae9231bedf3950379f89d4', 'name': 'P...",43.654895,-79.385052,"[{'label': 'display', 'lat': 43.65489527525682...",181,CA,Toronto,ON,Canada,"[112 Elizabeth St (at Dundas St W), Toronto ON...",112 Elizabeth St,at Dundas St W,M5G 1P5,NaN,57bcd3b7498e652a678d0378
3,Indigo,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.653515,-79.380696,"[{'label': 'display', 'lat': 43.65351471121164...",260,CA,Toronto,ON,Canada,"[220 Yonge St, Toronto ON M5B 2H1, Canada]",220 Yonge St,NaN,M5B 2H1,Downtown Yonge,4b2a6eb8f964a52012a924e3
4,Chatime 日出茶太,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",43.655542,-79.384684,"[{'label': 'display', 'lat': 43.65554164147378...",237,CA,Toronto,ON,Canada,"[132 Dundas St W (btwn Bay & University), Toro...",132 Dundas St W,btwn Bay & University,M5G 1C3,NaN,4e2284b11fc7c0ef9857d143
5,Textile Museum of Canada,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",43.654396,-79.386500,"[{'label': 'display', 'lat': 43.65439630500274...",230,CA,Toronto,ON,Canada,[55 Centre Avenue (University Ave. and Dundas ...,55 Centre Avenue,University Ave. and Dundas St W.,M5G 2H5,NaN,4ad4c05ef964a520e2f620e3
6,CF Toronto Eaton Centre,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",43.654447,-79.380952,NaN,263,CA,Toronto,ON,Canada,"[220 Yonge St (btwn Queen & Dundas), Toronto O...",220 Yonge St,btwn Queen & Dundas,M5B 2H1,"Downtown Toronto, Toronto, ON",4ad77a12f964a520260b21e3
7,Japango,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",43.655268,-79.385165,"[{'label': 'display', 'lat': 43.65526771691681...",222,CA,Toronto,ON,Canada,"[122 Elizabeth St. (at Dundas St. W), Toronto ...",122 Elizabeth St.,at Dundas St. W,M5G 1P5,NaN,4ae7b27df964a52068ad21e3
8,Old City Hall,"[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",43.652009,-79.381744,"[{'label': 'display', 'lat': 43.65200880087612...",240,CA,Toronto,ON,Canada,"[60 Queen Street West, Toronto ON M5H 1A1, Can...",60 Queen Street West,NaN,M5H 1A1,NaN,4ad4c05ef964a5208ef620e3
9,LUSH,"[{'id': '4bf58dd8d48988d10c951735', 'name': 'C...",43.653557,-79.380400,"[{'label': 'display', 'lat': 43.653557, 'lng':...",284,CA,Toronto,ON,Canada,"[220 Yonge St, Unit B215-A (in Toronto Eaton C...","220 Yonge St, Unit B215-A",in Toronto Eaton Centre,M5B 2H1,NaN,4bd0b30d41b9ef3b8fa0fae5


In [36]:
toronto_venues = getNearbyVenues(names=dataframe_filtered['name'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )

Downtown Toronto
Nathan Phillips Square
Poke Guys
Indigo
Chatime 日出茶太
Textile Museum of Canada
CF Toronto Eaton Centre
Japango
Old City Hall
LUSH
M Square Coffee Co
Marriott Downtown at CF Toronto Eaton Centre
Crepe Delicious
SEPHORA
Four Seasons Centre for the Performing Arts
MUJI
Sansotei Ramen 三草亭
Apple Eaton Centre
UNIQLO ユニクロ
DoubleTree by Hilton Hotel Toronto Downtown
Red Lobster
Bulldog On The Block
John & Sons Oyster House
Richmond Station
Rosalinda
Elgin And Winter Garden Theatres
Hudson's Bay
Yonge-Dundas Square
Solei Tanning Salon
The Keg Steakhouse + Bar - York Street


In [37]:
print(toronto_venues.shape)
toronto_venues.head()

(900, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.653232,-79.385296,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,Downtown Toronto,43.653232,-79.385296,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,Downtown Toronto,43.653232,-79.385296,Textile Museum of Canada,43.654396,-79.386500,Art Museum
3,Downtown Toronto,43.653232,-79.385296,Poke Guys,43.654895,-79.385052,Poke Place
4,Downtown Toronto,43.653232,-79.385296,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall


In [40]:
toronto_venues.groupby('Venue Category').count()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
American Restaurant,28,28,28,28,28,28
Art Gallery,2,2,2,2,2,2
Art Museum,11,11,11,11,11,11
Arts & Crafts Store,3,3,3,3,3,3
Asian Restaurant,7,7,7,7,7,7
Bakery,8,8,8,8,8,8
Bank,2,2,2,2,2,2
Bookstore,25,25,25,25,25,25
Brazilian Restaurant,1,1,1,1,1,1


In [41]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 67 uniques categories.


In [55]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Venue'] = toronto_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Venue,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Clothing Store,Coffee Shop,College Rec Center,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Diner,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Gastropub,Gluten-free Restaurant,Gym,Gym / Fitness Center,Hotel,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lingerie Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Music Venue,Neighborhood,New American Restaurant,Opera House,Park,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Speakeasy,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,University,Vegetarian / Vegan Restaurant
0,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Nathan Phillips Square,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Textile Museum of Canada,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Poke Guys,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Four Seasons Centre for the Performing Arts,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
toronto_grouped = toronto_onehot.groupby('Venue').mean().reset_index()
toronto_grouped

,Venue,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bookstore,Brazilian Restaurant,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Clothing Store,Coffee Shop,College Rec Center,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Diner,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Gastropub,Gluten-free Restaurant,Gym,Gym / Fitness Center,Hotel,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lingerie Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Music Venue,Neighborhood,New American Restaurant,Opera House,Park,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Smoke Shop,Speakeasy,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,University,Vegetarian / Vegan Restaurant
0,Aboveground Art Supplies,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adelaide Club Toronto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Apple Eaton Centre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Assembly Chef's Hall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Banh Mi Boys,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Beerbistro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Blaze Pizza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Bosk at Shangri-La,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Brick Street Bakery,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Bulldog On The Block,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
num_top_venues = 5

for hood in toronto_grouped['Venue']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Venue'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aboveground Art Supplies----
                 venue  freq
0  Arts & Crafts Store   1.0
1  American Restaurant   0.0
2  Monument / Landmark   0.0
3          Pizza Place   0.0
4                 Park   0.0


----Adelaide Club Toronto----
                  venue  freq
0  Gym / Fitness Center   1.0
1   American Restaurant   0.0
2   Monument / Landmark   0.0
3           Pizza Place   0.0
4                  Park   0.0


----Apple Eaton Centre----
                 venue  freq
0    Electronics Store   1.0
1  American Restaurant   0.0
2          Music Venue   0.0
3          Pizza Place   0.0
4                 Park   0.0


----Assembly Chef's Hall----
                 venue  freq
0           Food Court   1.0
1  American Restaurant   0.0
2          Music Venue   0.0
3          Pizza Place   0.0
4                 Park   0.0


----Banh Mi Boys----
                 venue  freq
0       Sandwich Place   1.0
1  American Restaurant   0.0
2          Music Venue   0.0
3          Pizza Place   0.0
4    

----Maman----
                 venue  freq
0                 Café   1.0
1  American Restaurant   0.0
2          Music Venue   0.0
3          Pizza Place   0.0
4                 Park   0.0


----Marriott Downtown at CF Toronto Eaton Centre----
                 venue  freq
0                Hotel   1.0
1  American Restaurant   0.0
2  Monument / Landmark   0.0
3          Pizza Place   0.0
4                 Park   0.0


----Mos Mos Coffee----
                 venue  freq
0                 Café   1.0
1  American Restaurant   0.0
2          Music Venue   0.0
3          Pizza Place   0.0
4                 Park   0.0


----NAMI----
                 venue  freq
0  Japanese Restaurant   1.0
1  American Restaurant   0.0
2  Monument / Landmark   0.0
3          Pizza Place   0.0
4                 Park   0.0


----Nathan Phillips Square----
                      venue  freq
0                     Plaza   1.0
1       American Restaurant   0.0
2                Poke Place   0.0
3                    Loung

In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Venue']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Venue'] = toronto_grouped['Venue']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head(100)

,Venue,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aboveground Art Supplies,Arts & Crafts Store,Vegetarian / Vegan Restaurant,Food Court,Concert Hall,Cosmetics Shop,Department Store,Diner,Electronics Store,Fast Food Restaurant,Furniture / Home Store
1,Adelaide Club Toronto,Gym / Fitness Center,Vegetarian / Vegan Restaurant,College Rec Center,Hotel,Gym,Gluten-free Restaurant,Gastropub,Furniture / Home Store,Food Court,Fast Food Restaurant
2,Apple Eaton Centre,Electronics Store,Vegetarian / Vegan Restaurant,College Rec Center,Hotel,Gym / Fitness Center,Gym,Gluten-free Restaurant,Gastropub,Furniture / Home Store,Food Court
3,Assembly Chef's Hall,Food Court,Vegetarian / Vegan Restaurant,College Rec Center,Hotel,Gym / Fitness Center,Gym,Gluten-free Restaurant,Gastropub,Furniture / Home Store,Fast Food Restaurant
4,Banh Mi Boys,Sandwich Place,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Concert Hall,Cosmetics Shop,Department Store,Diner,Electronics Store,Food Court,College Rec Center
5,Beerbistro,Gastropub,Vegetarian / Vegan Restaurant,College Rec Center,Hotel,Gym / Fitness Center,Gym,Gluten-free Restaurant,Furniture / Home Store,Food Court,Fast Food Restaurant
6,Blaze Pizza,Pizza Place,Fast Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Diner,Electronics Store,Vegetarian / Vegan Restaurant,Japanese Restaurant
7,Bosk at Shangri-La,Asian Restaurant,Vegetarian / Vegan Restaurant,Food Court,Concert Hall,Cosmetics Shop,Department Store,Diner,Electronics Store,Fast Food Restaurant,Furniture / Home Store
8,Brick Street Bakery,Bakery,Vegetarian / Vegan Restaurant,Food Court,Concert Hall,Cosmetics Shop,Department Store,Diner,Electronics Store,Fast Food Restaurant,Furniture / Home Store
9,Bulldog On The Block,Coffee Shop,College Rec Center,Hotel,Gym / Fitness Center,Gym,Gluten-free Restaurant,Gastropub,Furniture / Home Store,Food Court,Fast Food Restaurant
